In [20]:
import networkx as nx

input_path = "./data/knowledge_graph/knowledge_graph.graphml"
graph = nx.read_graphml(input_path)
graph

In [12]:
original_node_count = graph.number_of_nodes()
original_node_count

4048

In [13]:
graph.degree()

DiDegreeView({'- Organ: testes': 0, 'mitochondria': 1, 'germinal cells': 1, 'ovaries': 1, 'granules': 4, 'nucleus': 10, 'hyaloplasm': 3, 'Embryology': 1, 'cell': 2, 'Field of Study': 1, 'cytoplasm': 6, 'spongioplasm': 2, 'tissues': 1, 'somatic cells': 1, '- Node: monaster': 0, 'chromatin filaments': 2, 'man': 4, 'diaster': 1, 'nuclear membrane': 1, 'protoplasm': 3, 'centriole': 2, 'nuclear network': 2, 'Bonnet': 1, 'chromosomes': 8, 'Manual of Human Embryology': 3, 'attraction sphere': 1, 'Lehrbuch der Entwickelungsgeschichte': 1, 'centrosphere': 2, 'equator': 1, 'Keibel': 1, 'daughter chromosomes': 2, 'centrosome': 4, 'spindle': 2, 'Flemming': 1, '- BiologicalEntity: Yolk': 0, 'zona striata': 2, 'central position': 1, 'karyomitome': 1, 'cells': 4, 'ovum': 24, 'work': 1, 'new being': 1, 'cells of the follicle': 1, 'radially striated': 1, 'Uterus': 1, 'deutoplasm': 1, 'Uterine tube': 1, 'corona radiata': 2, 'Human ovum': 1, 'Henry Gray': 2, 'Germinal Vesicle': 2, '0.2 mm': 1, 'Follicle'

In [23]:
min_degree = 2
while True :
    nodes_to_remove = [
        node for node, degree in dict(graph.degree()).items() if degree < min_degree
    ]

    if not nodes_to_remove :
        break

    graph.remove_nodes_from(nodes_to_remove)
    

In [24]:
pruned_graph = graph.remove_nodes_from(nodes_to_remove)
print(graph)
# print(pruned_graph)

DiGraph with 976 nodes and 1799 edges


In [25]:
for node in graph.nodes() :
    degree = graph.degree(node)
    graph.nodes[node]["degree_weight"] = degree

In [26]:
import os
output_path = "./data/knowledge_graph/pruned_knowledge_graph.graphml"
output_dir = os.path.dirname(output_path)
if output_dir :
    os.makedirs(output_dir, exist_ok=True)

nx.write_graphml(graph, output_path, infer_numeric_types=True)

# 가지치기

In [ ]:
import os
import re
import time
import nltk
import json
import torch
import chromadb
import numpy as np
import networkx as nx
import nltk.downloader
from datetime import datetime
from pykeen.models import ComplEx
from pykeen.pipeline import pipeline
from transformers import AutoTokenizer
from nltk.tokenize import sent_tokenize
from model_loader_folder.config import *
from pykeen.triples import TriplesFactory
from chromadb.utils import embedding_functions
from pykeen.optimizers import AdamW as PyKeenAdamW
from typing import List, Dict, Any, Tuple, Optional, Set
from sentence_transformers import CrossEncoder, SentenceTransformer

class CustomEmbeddingFunction(embedding_functions.EmbeddingFunction):
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model
    
    def __call__(self, texts):
        return self.embedding_model.encode(texts, convert_to_numpy=True).tolist()
    
nltk.download("punkt")
nltk.download("punkt_tab")

generation_loader = generation_loader

class QASystem:
    def __init__(self, 
                 graphml_path: str, 
                 md_path: str,
                 vector_db_path: str = "./chroma_db_split", 
                 similarity_threshold: float = 0.5,
                 chunk_token_threshold: int = 250,
                 embedding_model_path: str = "./model/embedding/bge-m3",
                 m_hop_depth: int=3,
                 load_embedding_in_4bit: bool=False,
                 load_embedding_in_8bit: bool=False):
        self.graphml_path = graphml_path
        self.md_path = md_path
        self.similarity_threshold = similarity_threshold
        self.chunk_token_threshold = chunk_token_threshold
        self.llm_loader = None
        self.m_hop_depth = m_hop_depth
        
        self.tokenizer = AutoTokenizer.from_pretrained("cross-encoder/ms-marco-TinyBERT-L-2-v2")
        # self.tokenizer = AutoTokenizer.from_pretrained(embedding_model_path)
        # self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")
        self.reranker = CrossEncoder("colbert-ir/colbertv2.0")

        if load_embedding_in_4bit and load_embedding_in_8bit :
            raise ValueError("Cannot load embedding model in both 4bit and 8bit")
        
        if load_embedding_in_4bit :
            self.embedding_model = SentenceTransformer(
                embedding_model_path,
                model_kwargs={"load_in_4bit": True}
            )
        elif load_embedding_in_8bit :
            self.embedding_model = SentenceTransformer(
                embedding_model_path,
                model_kwargs={"load_in_8bit": True}
            )
        else :
            self.embedding_model = SentenceTransformer(embedding_model_path)

        self.custom_embedding_function = CustomEmbeddingFunction(self.embedding_model)

        self.graph = nx.read_graphml(graphml_path)
        self.node_name_map = {self._preprocess_text(node): node for node in self.graph.nodes()}
        self.client = chromadb.PersistentClient(path=vector_db_path)
        
        self.entity_collection = self.client.get_or_create_collection(name="entities_split", embedding_function=self.custom_embedding_function)
        self.relation_collection = self.client.get_or_create_collection(name="relations_split", embedding_function=self.custom_embedding_function)
        self.chunk_collection = self.client.get_or_create_collection(name="chunks", embedding_function=self.custom_embedding_function)
        self.entity_relation_extraction_prompt_template = """
            Extract entities and their relations from the following sentence.

            **Entities** should be **unique nouns or concepts**, extracted as **noun phrases** whenever possible. Identify **concrete objects or concepts** rather than complex activities or phenomena as entities.

            **Relations** should clearly describe the connection between two entities, preferring **reusable predicate verbs** for a knowledge graph. Use **concise verbs** or clear, hyphenated forms like **'part_of' or 'includes'**.

            Output the result **only in the following JSON format**, with no other explanations or text:

            ```json
            {{
                "entities": [
                    {{"name": "Entity1", "type": "Type (e.g., Organ, System, Substance, Function, Disease)"}},
                    {{"name": "Entity2", "type": "Type"}}
                ],
                "relations": [
                    {{"head": "Entity1", "relation": "Relation_Type (e.g., part_of, causes)", "tail": "Entity2"}},
                    {{"head": "Entity3", "relation": "generates", "tail": "Entity4"}}
                ]
            }}

            sentence : "{text_to_analyze}"
            JSON result :
        """

        self._initialize_vector_db()

    def _preprocess_text(self, text: str) -> str:
        return text.upper().replace(' ', '_')

    def _create_chunks_from_text(self, text: str, page_num: str) -> List[Dict[str, Any]] :
        chunks = []
        paragraphs = re.split("\n\n+", text)
        for para in paragraphs :
            para = para.strip()
            if not para :
                continue

            para_tokens = self.tokenizer.tokenize(para)

            if len(para_tokens) <= self.chunk_token_threshold :
                chunks.append({"document": para, "metadata": {"source_page": page_num}})
            else :
                sentences = sent_tokenize(para)
                current_chunk_sentences = []
                current_chunk_tokens = 0

                for sentence in sentences :
                    sentence_tokens = self.tokenizer.tokenize(sentence)

                    if current_chunk_tokens + len(sentence_tokens) > self.chunk_token_threshold and current_chunk_sentences :
                        chunk_text = " ".join(current_chunk_sentences)
                        chunks.append({"document": chunk_text, "metadata": {"source_page": page_num}})
                        current_chunk_sentences = [sentence]
                        current_chunk_tokens = len(sentence_tokens)
                    else :
                        current_chunk_sentences.append(sentence)
                        current_chunk_tokens += len(sentence_tokens)

                if current_chunk_sentences :
                    chunk_text = " ".join(current_chunk_sentences)
                    chunks.append({"document": chunk_text, "metadata": {"source_page": page_num}})

        return chunks

    def _initialize_vector_db(self):
        if self.entity_collection.count() == 0:
            nodes_to_add = []
            unique_nodes = set()
            for node, data in self.graph.nodes(data=True):
                processed_node = self._preprocess_text(node)
                if processed_node not in unique_nodes :
                    metadata = {k: str(v) for k, v in data.items()}
                    metadata['original_name'] = node
                    nodes_to_add.append({'id': processed_node, 'document': node, 'metadata': metadata})
                    unique_nodes.add(processed_node)
            
            if nodes_to_add:
                ids = [item['id'] for item in nodes_to_add]
                documents = [item['document'] for item in nodes_to_add]
                metadatas = [item['metadata'] for item in nodes_to_add]
                self.entity_collection.add(ids=ids, documents=documents, metadatas=metadatas)

        if self.relation_collection.count() == 0:
            edges_to_add = []
            unique_processed_relations = set()
            for u, v, data in self.graph.edges(data=True):
                relation_type = data.get('type')
                if relation_type:
                    processed_relation = self._preprocess_text(relation_type)
                    if processed_relation not in unique_processed_relations :
                        metadata = {'original_name': relation_type}
                        edges_to_add.append({'id': processed_relation, 'document': processed_relation, 'metadata': metadata})
                        unique_processed_relations.add(processed_relation)

            if edges_to_add:
                ids = [item['id'] for item in edges_to_add]
                documents = [item['document'] for item in edges_to_add]
                metadatas = [item['metadata'] for item in edges_to_add]
                self.relation_collection.add(ids=ids, documents=documents, metadatas=metadatas)
    
    def _extract_entities_relations(self, question) :
        prompt = self.entity_relation_extraction_prompt_template.format(text_to_analyze=question)
        raw_llm_output = self._call_llm_generate(prompt)

        try :
            json_start = raw_llm_output.find("{")
            # json_end = raw_llm_output.rfind("}") + 1
            if json_start != -1 :
                decoder = json.JSONDecoder()
                extracted_data, _ = decoder.raw_decode(raw_llm_output[json_start:])
                return extracted_data.get("entities", []), extracted_data.get("relations", [])
            else :
                print(f"LLM 답변에서 유효한 JSON 형태를 찾을 수 없음 : {raw_llm_output}")
                return [], []
            
        except json.JSONDecodeError as e :
            print(f"개체 추출 과정에서 JSON 디코딩 오류 발생: {e}")
            print(f"오류 발생 원문: {raw_llm_output}")
            return [], []
    
    def _find_pages_from_entities(self, expanded_entities: List[Dict[str, Any]]) -> Set[str]:
        query_texts = [e['name'] for e in expanded_entities if 'name' in e]
        if not query_texts:
            return set()

        # 유사 엔티티 탐색
        entity_results = self.entity_collection.query(
            query_texts=query_texts,
            n_results=5,
            include=["metadatas", "distances"]
        )
        # print(f"벡터DB 검색 결과 : {entity_results.get('distances')}")
        # print(f"쿼리 : {query_texts}")
        # print(f"검색 결과 : {entity_results}")

        similar_entity_names = set()
        if entity_results.get('distances'):
            for i, dists in enumerate(entity_results['distances']):
                for j, dist in enumerate(dists):
                    if dist <= self.similarity_threshold:
                        meta = entity_results['metadatas'][i][j]
                        similar_entity_names.add(meta['original_name'])
        
        # print(f"유사도 필터링 후 엔티티 : {similar_entity_names}")
        page_numbers = set()
        for entity_name in similar_entity_names:
            standarized_name = self._preprocess_text(entity_name)
            if standarized_name in self.node_name_map :
                original_case_node_name = self.node_name_map[standarized_name]
                node_data = self.graph.nodes[original_case_node_name]
                # print(f"node_data : {node_data}")
                pages_str = node_data.get("source_page")

                # print(f"{entity_name} 노드 데이터 : {node_data}")
                if pages_str :
                    for page in pages_str.split(',') :
                        if page.strip() :
                            page_numbers.add(page.strip())
            # else :
            #     print(f"{entity_name}을 그래프에서 찾을 수 없음")
        return page_numbers
    
    def _retrieve_and_rerank_context(self, question: str, page_numbers: Set[str], top_k_rerank: int = 5) -> List[Dict[str, Any]]:
        if not page_numbers:
            return []

        all_md_content = ""
        for md_file in os.listdir(self.md_path):
            if md_file.endswith(".md"):
                with open(os.path.join(self.md_path, md_file), 'r', encoding='utf-8') as f:
                    all_md_content += f.read() + "\n\n"

        candidate_chunks = []
        for page_num in page_numbers:
            pattern = re.compile(rf"####\s+Page\s+{re.escape(page_num)}\b(.*?)(?=####\s+Page|\Z)", re.S)
            match = pattern.search(all_md_content)
            
            # print(f"Searching for page: '{page_num}'")
            # if not match:
            #     print(f"-> Page '{page_num}' NOT FOUND in markdown content.")
            # else:
            #     print(f"-> Page '{page_num}' FOUND.")

            if match:
                page_content = match.group(1).strip()
                chunks = self._create_chunks_from_text(page_content, page_num)
                candidate_chunks.extend(chunks)
        
        if not candidate_chunks:
            return []
            
        rerank_pairs = [(question, chunk['document']) for chunk in candidate_chunks]
        if not rerank_pairs:
            return []

        scores = self.reranker.predict(rerank_pairs)

        reranked_results = []
        for score, chunk in zip(scores, candidate_chunks):
            chunk["rerank_score"] = score
            reranked_results.append(chunk)

        reranked_results.sort(key=lambda x: x["rerank_score"], reverse=True)

        return reranked_results[:top_k_rerank]

    def _build_llm_prompt(self, question: str, context: str) -> str:
        prompt = f"""
        You are a helpful assistant who answers questions based on the provided context.
        You MUST cite the source page number for every piece of information you use.

        **Instructions:**
        1. Answer the user's question clearly and concisely using ONLY the provided context and knowledge graph information.
        2. For every statement, you MUST provide the source page number in parentheses, like this: (Page XX).
        3. If a single piece of information is supported by multiple pages, cite all of them: (Page X, Y, Z).
        4. If no context is available, state that you are answering based on the graph structure alone.

        **Example of a GOOD answer:**
        The ductus arteriosus degenerates into the ligamentum arteriosum after birth(page 360). This is a normal physiological change that happens post-delivery(page 361).

        **Example of a BAD answer:** -> (This is a bad answer because it lacks the mandatory citation)
        The ductus arteriosus becomes the ligamentum arteriosum.

        ---
        **Context:**
        {context}
        ---
        **Question:**
        {question}
        ---
        **Answer:**
        """
        return prompt.strip()
    
    def _call_llm_generate(self, prompt: str) -> str:
        if self.llm_loader:
            if hasattr(self.llm_loader, "tokenizer") and hasattr(self.llm_loader, "model"):
                tokenizer = self.llm_loader.tokenizer
                model = self.llm_loader.model

                input_ids = tokenizer.encode(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)

                output = model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    max_new_tokens=500,
                    temperature=0.0,
                    do_sample=False,
                    top_p=0.85,
                    repetition_penalty=1.2,
                    early_stopping=True,
                    num_beams=3,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )
                generated_ids = output[0][input_ids.shape[-1]:]
                raw_answer = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
                return raw_answer
            else:
                raw_answer = self.llm_loader.generate(prompt)
                return raw_answer
        else:
            print("generation_loader가 로드되지 않음")
            return "LLM 로더가 설정되지 않았습니다."

    def _m_hop_expand_entities(self, initial_entity_names: List[str], max_hops: int) -> Set[str] :
        if not self.graph :
            print("지식 그래프가 로드되지 않음")
            return set(initial_entity_names)
        
        expanded_entity_names = set()
        visited_nodes = set()

        queue = []
        
        for name in initial_entity_names :
            standarized_name = self._preprocess_text(name)

            if standarized_name in self.node_name_map :
                original_case_name = self.node_name_map[standarized_name]
                if original_case_name not in visited_nodes :
                    queue.append((original_case_name, 0))
                    visited_nodes.add(original_case_name)
                    expanded_entity_names.add(original_case_name)
        while queue :
            current_node_name, current_hop = queue.pop(0)

            if current_hop < max_hops :
                for neighbor_node_name in self.graph.neighbors(current_node_name) :
                    if neighbor_node_name not in visited_nodes :
                        visited_nodes.add(neighbor_node_name)
                        expanded_entity_names.add(neighbor_node_name)
                        queue.append((neighbor_node_name, current_hop + 1))

        return expanded_entity_names

    def _expand_entities(self, initial_entities: List[Dict[str, Any]], m_hop_depth: int) -> List[Dict[str, Any]]:
        if not initial_entities:
            return []

        initial_entity_names = [e['name'] for e in initial_entities]
        prompt = f"""
        Given the following list of medical or anatomical entities, provide a list of related or synonymous entities.
        This will be used to improve search recall in a knowledge base.
        Focus on providing closely related concepts, components, or alternative names.
        Provide the output as a simple comma-separated list, without numbers or bullets.

        Initial Entities: {', '.join(initial_entity_names)}

        Expanded Entities:
        """

        raw_llm_output = self._call_llm_generate(prompt.strip())

        llm_expanded_entity_names = {name.strip() for name in raw_llm_output.split(',') if name.strip()}
        graph_expanded_entity_names = self._m_hop_expand_entities(initial_entity_names, m_hop_depth)
        
        all_entity_names = set(initial_entity_names)
        all_entity_names.update(llm_expanded_entity_names)
        all_entity_names.update(graph_expanded_entity_names)

        final_entities = [{'name': name} for name in all_entity_names]

        # print(f"질문에서 추출된 엔티티: {initial_entity_names}")
        # print(f"LLM 확장 엔티티 : {llm_expanded_entity_names}")
        # print(f"M-hop 탐색 엔티티 : {list(graph_expanded_entity_names)}")
        # print(f"최종 확장 엔티티 : {[e['name'] for e in final_entities]}")

        return final_entities

    def generate_response(self, question: str, m_hop_depth: int) -> Tuple[str, str]:
        # 질문 엔티티 추출
        initial_entities, _ = self._extract_entities_relations(question)
        if not initial_entities:
            return "질문에서 유효한 엔티티를 추출할 수 없습니다.", ""

        # llm 기반 엔티티 확장 & m-hop 깊이 확장
        expanded_entities = self._expand_entities(initial_entities, m_hop_depth=m_hop_depth)
        
        # 확장 엔티티 유사도 검색 후 페이지 파싱
        page_numbers = self._find_pages_from_entities(expanded_entities)
        if not page_numbers :
            return "페이지를 찾을 수 없음", ""
        print(f"찾은 페이지 : {page_numbers}")

        # 파싱된 페이지 원문 청크 검색 & 재정렬
        reranked_chunks = self._retrieve_and_rerank_context(question, page_numbers, top_k_rerank=5)
        print(f"reranked_chunks : {reranked_chunks}")
        if not reranked_chunks:
            return "관련 페이지는 찾았으나, 질문과 직접적으로 연관된 문맥이 없음", ""

        final_context_parts = []
        current_len = 0

        if hasattr(self.llm_loader, 'tokenizer') and self.llm_loader.tokenizer is not None:
            print("LLM 로더의 특정 토크나이저를 사용하여 길이를 계산합니다.")
            llm_tokenizer = self.llm_loader.tokenizer
            max_len = getattr(llm_tokenizer, 'model_max_length', 4096) - 500

            base_prompt = self._build_llm_prompt(question, "")
            base_prompt_len = len(llm_tokenizer.tokenize(base_prompt))
            current_len += base_prompt_len

            for chunk in reranked_chunks:
                page_num = chunk['metadata'].get('source_page', 'N/A')
                context_snippet = f"... {chunk['document']} ... (출처: Page {page_num})"
                chunk_token_len = len(llm_tokenizer.tokenize(context_snippet))

                if current_len + chunk_token_len <= max_len:
                    final_context_parts.append(context_snippet)
                    current_len += chunk_token_len
                else:
                    break
        else:
            print("범용 토크나이저를 사용하여 길이를 근사치로 계산합니다. (Ollama 등)")
            proxy_tokenizer = self.tokenizer  
            max_len = 2048 - 500

            for chunk in reranked_chunks:
                page_num = chunk['metadata'].get('source_page', 'N/A')
                context_snippet = f"... {chunk['document']} ... (출처: Page {page_num})"
                chunk_token_len = len(proxy_tokenizer.tokenize(context_snippet))

                if current_len + chunk_token_len <= max_len:
                    final_context_parts.append(context_snippet)
                    current_len += chunk_token_len
                else:
                    break

        if not final_context_parts:
            return "관련 정보를 찾았으나, 모델의 입력 길이 제한으로 인해 컨텍스트를 구성할 수 없습니다.", ""

        context = "\n\n".join(final_context_parts)

        prompt = self._build_llm_prompt(question, context)

        answer = self._call_llm_generate(prompt)
        return answer, context

def save_results_to_file(question: str, answer: str, context: str, output_dir: str, file_index: int):
    os.makedirs(output_dir, exist_ok=True)

    timestamp = datetime.now().strftime("%H%M%S_%f")
    file_name = f"result_{file_index}_{timestamp}.txt"
    file_path = os.path.join(output_dir, file_name)
    
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(f"[질문]\n{question}\n\n")
        f.write(f"[답변]\n{answer}\n\n")
        f.write(f"[근거]\n{context}\n\n")

if __name__ == '__main__':
    qa_system = QASystem(
        graphml_path="./data/knowledge_graph/knowledge_graph.graphml",
        md_path="./data/split_file/anatomy/",
        m_hop_depth=1,
        load_embedding_in_4bit=True
    )
    
    qa_system.llm_loader = generation_loader
    
    questions = [
        ############## 1_Embryology.md
        "What are the two essential components of a higher organism cell as defined in the text?", # 7페이지
        "Describe the four main phases of indirect cell division (karyokinesis) as outlined in the text.", # 7페이지
        "What is the primary role of the yolk-sac in the embryo's early development?", # 20페이지
        "How does the embryo separate from the yolk-sac, and what does the enclosed part of the yolk-sac form?", # 19페이지
        "What significant developments occur in a human embryo during the Second Week?", # 33페이지
        "What are the key characteristics of the human embryo by the end of the Third Week?", # 33페이지
        
        ############## 2_Osteology.md
        "What are the three groups into which the cells of a primitive segment differentiate, and what do they form?", # 38페이지
        "How is each vertebral body formed from primitive segments during development?", # 38페이지
        "What are the sphenoidal air sinuses, and where are they located within the sphenoid bone?", # 88페이지
        "Describe the sphenoidal rostrum and its articulation.",# 88
        "What is the tibia, and where is it located in the human leg?", # 158
        "Describe the superior articular surface of the tibia's upper extremity.", # 158

        ############## 3_Syndesmology.md
        "What are joints or articulations, and how are immovable joints characterized?", # 174
        "How does the articular lamella differ from ordinary bone tissue?", # 174
        "Where is the synovial membrane located in relation to the glenoid cavity and humerus, and how does it interact with the Biceps brachii tendon?", # 207
        "List some of the bursae located near the shoulder-joint and specify which ones communicate with the synovial cavity.", # 207
        "What is the function of the plantar calcaneonavicular ligament, and what condition results if it yields?", # 236
        "How are the navicular bone and the three cuneiform bones connected, and what type of movement do they permit?", # 236

        ############## 4_Myology.md
        "How does the nervous system serve as an indicator for the origin and migration paths of developing muscles, despite not influencing muscle differentiation?", # 250
        "Describe the structural components of striped or voluntary muscle, from bundles to individual fibers.", # 250
        "What is the triangular ligament and where is it located?", # 290
        "What structures perforate the superficial layer (inferior fascia) of the urogenital diaphragm?", # 290
        "Where does the Extensor digitorum longus muscle originate, and what structures are located between it and the Tibialis anterior?", # 322
        "What is the Peronæus tertius, and where is it inserted?", # 322

        ############## 5_Angiology.md
        "What are the main characteristics of the middle coat (tunica media) of arteries, and how does its composition vary with vessel size?", # 334
        "Describe the composition and variations of the external coat (tunica adventitia) in arteries.", # 334
        "How do the Vitelline Veins develop into parts of the portal and hepatic veins?", # 345
        "What happens to the Umbilical Veins during embryonic development and after birth?", # 345
        "What are the three phases of a cardiac cycle and what happens during each?", # 358
        "What are the main peculiarities observed in the fetal heart's vascular system?" # 359
    ]   
    today = datetime.now()
    folder_name = f"{today.month}월{today.day}일"
    output_dir = os.path.join("./result", "knowledge_graph", folder_name, "m-3")
    total_time = 0
    for i, q in enumerate(questions):
        print(f"질문: {q}")
        start_time = time.time()
        response, context = qa_system.generate_response(q, m_hop_depth=3)
        end_time = time.time()
        elapse_time = end_time - start_time
        total_time += elapse_time
        print("=" * 30)
        print(f"답변: {response}\n\n\n")
        print("=" * 30)
        save_results_to_file(q, response, context, output_dir, i + 1)
    avg_time = total_time / len(questions)
    print("평균 답변 시간 : %.2f" % avg_time)

/opt/conda/envs/lang_project/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.99it/s]
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at colbert-ir/colbertv2.0 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesC

ValueError: Due to a serious vulnerability issue in `torch.load`, even with `weights_only=True`, we now require users to upgrade torch to at least v2.6 in order to use the function. This version restriction does not apply when loading files with safetensors.
See the vulnerability report here https://nvd.nist.gov/vuln/detail/CVE-2025-32434

: 